## 12/11
- 肥満、高脂血症、高血糖、脂質異常（MetSにおいて）を判定
- メタボの判定


In [1]:
import master_code as mc

In [ ]:
import numpy as np
import pandas as pd
main = pd.read_csv('main.csv', encoding = 'cp932')

In [ ]:
df = main.copy()
metS_facter = [
    ['性別', '腹囲'],               # 腹囲
    ['収縮機血圧', '拡張期血圧'],     # 血圧
    ['空腹時血糖', 'HbA1c'],        # 血糖
    ['中性脂肪', 'HDL']             # 脂質
]

def mets_judge(df, metS_facter):

    df['腹囲_'] = (((df[metS_facter[0][0]] == 1) & (df[metS_facter[0][1]] >= 85) |   # 男性
                (df[metS_facter[0][0]] == 0) & (df[metS_facter[0][1]] >= 90))     # 女性
                .where((df[metS_facter[0][0]].notnull()) & (df[metS_facter[0][1]].notnull()), np.nan)
                ) 
    df['血圧_'] = (((df[metS_facter[1][0]] >= 130) |    # 収縮期血圧
                (df[metS_facter[1][1]] >= 85))       # 拡張期血圧
                .where((df[metS_facter[1][0]].notnull()) & (df[metS_facter[1][1]].notnull()), np.nan)
                )
    df['血糖_'] = (((df[metS_facter[2][0]] >= 110) |    # 空腹時血糖
                (df[metS_facter[2][1]] >= 5.6))             # HbA1c
                .where((df[metS_facter[2][0]].notnull()) & (df[metS_facter[2][1]].notnull()), np.nan)
                )
    df['脂質_'] = (((df[metS_facter[3][0]] >= 150) |    # 中性脂肪
                (df['HDL'] <= 40))                     # HDL
                .where((df[metS_facter[3][0]].notnull()) & (df[metS_facter[3][1]].notnull()), np.nan)
                )
    
    # nanの変換
    # df['腹囲'] = df['腹囲'].where((~df['腹囲'].notnull()) | (~df['性別'].notnull()), np.nan)
    # df['血圧'] = df['血圧'].where((~df['収縮機血圧'].notnull()) | (~df['拡張期血圧'].notnull()), np.nan)
    # df['血糖'] = df['血糖'].where((~df['空腹時血糖'].notnull()) | (~df['HbA1c'].notnull()), np.nan)
    # df['脂質'] = df['脂質'].where((~df['中性脂肪'].notnull()) | (~df['HDL'].notnull()), np.nan)

    # 判定
    df['MetS'] = (df['腹囲_'] == True) & ((df['血圧_'] + df['血糖_'] + df['脂質_'] >= 2))

    df_metS = df[df['MetS'] == True]
    df_non_metS = df[df['MetS'] == False]
    return df_metS, df_non_metS

In [ ]:
# 実行
df_metS, df_non_metS = mets_judge(df, metS_facter)
print(df_metS)
print(df_non_metS)